In [12]:

import pandas as pd
    
import cx_Oracle

f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()

host = ''
port = 1
service = '1'
cx_Oracle.init_oracle_client(lib_dir= path) # use this is service isn't initialised

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()

query = """
select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_EMPLOYER" "Employer_Name", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_END_DATE" "End_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_INDUSTRY" "Industry", "WBC_OES_APPLICATION_D"."REASON_FOR_LEAVING_EMPLOYMENT" "Reason_for_Leaving", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_START_DATE" "Start_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_JOB_TITLE" "Work_Title"
 from ((
select "WBC_OES_APPLICATION_F"."TAAAPA_WID" "TAAAPA_WID", "WBC_OES_APPLICATION_F"."CASE_WID" "CASE_WID"
 from "CDW"."WBC_OES_APPLICATION_F" "WBC_OES_APPLICATION_F"
) "WBC_OES_APPLICATION_F" INNER JOIN "CDW"."WBC_CASE_D" "WBC_CASE_D" on "WBC_OES_APPLICATION_F"."CASE_WID"="WBC_CASE_D"."CASE_WID") LEFT OUTER JOIN "CDW"."WBC_OES_APPLICATION_D" "WBC_OES_APPLICATION_D" on "WBC_OES_APPLICATION_F"."TAAAPA_WID"="WBC_OES_APPLICATION_D"."TAAAPA_WID"



"""

df = pd.read_sql(query, con=conn)


In [13]:
df = df.loc[df['Employer_Name'].notnull() == True,]
df = df.head(1000)
pd.set_option('display.max_rows', 100)
df.head(80)


,Case_Number,Employer_Name,End_Date,Industry,Reason_for_Leaving,Start_Date,Work_Title
0,0,TELUS,2019-04-02 00:00:00,Telecommunications,None,2003-09-09 00:00:00,Network Technician
1,0,Aspiral Youth Association,None,Downtown Activity Centre,"I worked casual, filling in for Sandy when she...",None,Office assistant
2,0,super 8 motel,2018-08-15 00:00:00,hotel,Injury at work with surgery in December 2018 ...,2018-04-02 00:00:00,maintenance / security
4,0,Embers,None,Construction,None,2019-04-02 00:00:00,General Labourer
5,0,Akita Drilling LTD,2019-01-17 00:00:00,Oil Feild,"Very slow, Laid off, no work available, I am s...",1999-10-20 00:00:00,"Rig Manager, Safety Coach"
6,0,Ajit newspaper,None,Publication,"Moved to Canada, used to work for this company.",None,DTP operator
7,0,Vince Hariott construction,None,construction site,I have done some trades work in construction o...,None,construction worker
8,0,loblaws,None,None,None,2003-09-07 00:00:00,cashier
9,0,Lakeside Hotel And Casino,2018-09-23 00:00:00,Hospitlility,"Attacked while on vacation in Mexico , later i...",2016-05-04 00:00:00,Janiter
10,0,Fiamo restaurant and Fifth street Bar and Grill,None,Food,None,None,Dishwasher


In [14]:
from spellchecker import SpellChecker
import re
spell = SpellChecker()
df = df[df['Employer_Name'].notnull()]
for index, row in df.iterrows():

    x = re.findall(r"[\w']+", row['Employer_Name'])
    x = [spell.correction(w) for w in x]
    df.at[index, 'Employer_Name'] = ' '.join(x)

In [53]:
import requests
#var = "safe on foods"
#query = {'q':'"'+ var+'"~'}
#print(query)
#response = requests.get("https://orgbook.gov.bc.ca/api/v4/search/fuzzy", params = query)
n=0
dd = df
dd['response'] = ''
for index, row in dd.iterrows():
    n=n+1
    if n%10 == 0:
        print(n)
    if len(row['Employer_Name'].split()) == 1:    
        response = requests.get('https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+row['Employer_Name']+'~')
        l=[]
        p = len(response.json())
        if p >0:
            for x in range(p):

                l.append(response.json()[x]['topic_names'])
        dd.at[index, 'response'] = l
        
    else:
        l ='~ AND '.join(row['Employer_Name'].split())
        t = '(' + l + '~)'
        response = requests.get('https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+t)
        l=[]
        print('https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+t)
        print(response.json())
        p = len(response.json())
        
        
        if p >0:
            for x in range(p):
                print(p)
                l.append(response.json()[x]['topic_names'])
            
        
        else:
#             pass
            l ='~ OR '.join(row['Employer_Name'].split())
            t = '(' + l + '~)'
            response = requests.get('https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+t)
            l=[]
            print('https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+t)
            print(response.json())
            p = len(response.json())


            if p >0:
                for x in range(p):
                    print(p)
                    l.append(response.json()[x]['topic_names'])
        dd.at[index, 'response'] = l

        
    #if n > 100:
       # break

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(spiral~ AND Youth~ AND Association~)
[{'id': 2164372, 'topic_source_id': 'S0044465', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['ASPIRAL YOUTH PARTNERS ASSOCIATION', '853664530'], 'topic_inactive': False, 'topic_revoked': False}]
1
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(super~ AND 8~ AND motel~)
[{'id': 1162580, 'topic_source_id': 'BC1139809', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['SUPER 7 MOTEL LTD.', '787130681'], 'topic_inactive': True, 'topic_revoked': False}]
1
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Akita~ AND Drilling~ AND led~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Akita~ OR Drilling~ OR led~)
[{'id': 1293908, 'topic_source_id': 'A0036432', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['AKITA DRILLING LTD.', '134056142'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1324191, 'topic_source_id': 'A0064259', 'topic_issuer_id': 1, 'topic_type_

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(siamo~ AND restaurant~ AND and~ AND Fifth~ AND street~ AND Bar~ AND and~ AND Grill~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(siamo~ OR restaurant~ OR and~ OR Fifth~ OR street~ OR Bar~ OR and~ OR Grill~)
[{'id': 672309, 'topic_source_id': 'BC0652319', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['FIFTH STREET BAR & WOODFIRE GRILL LTD.', '893295295'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1047882, 'topic_source_id': 'BC1027224', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['ORCA GRILLHOUSE AND RESTAURANT LTD.', '816308795'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 802580, 'topic_source_id': 'BC0785862', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['CABANA GRILLE RESTAURANTS INC.', '857473391'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 767645, 'topic_source_id': 'BC0749372', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['THE ATLA

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(self~ OR Decade~ OR resources~ OR Mountain~ OR Boy~ OR Minerals~)
[{'id': 2209106, 'topic_source_id': 'FM0795914', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['RAW MOUNTAIN MINERALS & RESOURCES', '752820936'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1946252, 'topic_source_id': 'FM0611110', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BEYOND DECADENCE CONFECTIONS', '808083133'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 314173, 'topic_source_id': 'BC0307323', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['MINERAL MOUNTAIN RESOURCES LTD.'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 802871, 'topic_source_id': 'FM0554259', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['DECADENT DWELLINGS'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1758407, 'topic_source_id': 'FM0401097', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Tower~ AND Planning~ AND Co~ AND led~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Tower~ OR Planning~ OR Co~ OR led~)
[{'id': 982208, 'topic_source_id': 'BC0963644', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['PLANNINGWORKS CONSULTING INC.', '826360935'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 431830, 'topic_source_id': 'BC0421261', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['LEADAIR COOLING TOWERS LTD.'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 2214743, 'topic_source_id': 'FM0801197', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['PLANNINGBYLISA FINANCIAL SOLUTIONS', '769389537'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1396365, 'topic_source_id': 'FM0004792', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['LEADERSHIP DYNAMICS PLANNING'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1982075, 'topic_source_id': 'FM

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(A~ AND i~ AND Fruit~ AND Growers~ AND led~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(A~ OR i~ OR Fruit~ OR Growers~ OR led~)
[{'id': 1715220, 'topic_source_id': 'FM0354691', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['FRUITVALE GROWERS'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 852512, 'topic_source_id': 'FM0492201', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['SUPER FRUITHANA FRUITS & VEGETABLES'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1390051, 'topic_source_id': 'CP0000087', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BOSWELL FRUIT GROWERS'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 395813, 'topic_source_id': 'BC0386363', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['A & P FRUIT GROWERS LTD.', '129776548'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 342270, 'topic_source_id': 'BC0334519', 'topic_issuer_id

...

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Pacific~ OR Community~ OR Church~)
[{'id': 2129036, 'topic_source_id': 'S0008367', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['COMMUNITY CHURCHES OF AMERICA SOCIETY'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 2126724, 'topic_source_id': 'S0005998', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['COMMUNITY CHURCH OF CHRIST (COMMUNITY CHRISTIAN CHURCH)', '106961378'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1865325, 'topic_source_id': 'FM0518494', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['JL PACIFICO COMMUNITY SOLUTIONS'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2133500, 'topic_source_id': 'S0012933', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['CALVARY COMMUNITY CHURCH'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 2134036, 'topic_source_id': 'S0013481', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['CHRIST COM

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(The~ OR View~ OR bonita~)
[{'id': 1895420, 'topic_source_id': 'FM0551932', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BONITAS BISTRO'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1718086, 'topic_source_id': 'FM0357726', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BONITAS WINERY'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1901111, 'topic_source_id': 'FM0558372', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BONITAS BISTRO'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 605701, 'topic_source_id': 'BC0588660', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BONITAS INTERACTIVE LTD.', '866258270'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 687329, 'topic_source_id': 'BC0666635', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['BONITAS INVESTMENTS LTD.', '892252503'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Creativity~ AND and~ AND Innovation~ AND Centre~ AND put~ AND Limited~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(Creativity~ OR and~ OR Innovation~ OR Centre~ OR put~ OR Limited~)
[{'id': 1888760, 'topic_source_id': 'FM0544503', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['CREATIVITALITY'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2339720, 'topic_source_id': 'FM0848738', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['CREATIVITUS'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 1901090, 'topic_source_id': 'FM0558348', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['THE CREATIVITY CENTER AND GALLERY'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2187114, 'topic_source_id': 'S0068248', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['THE ALLIANCE OF CREATIVITY-INNOVATION-STARTUPS SOCIETY', '789595485'], 'topic_inactive': False, 'topic_revoked':

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(tow~ AND Tun~ AND Le~ AND Lum~ AND Society~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(tow~ OR Tun~ OR Le~ OR Lum~ OR Society~)
[{'id': 2136883, 'topic_source_id': 'S0016396', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['TSOW-TUN LE LUM SOCIETY', '108102005'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2148919, 'topic_source_id': 'S0028656', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['TOWNSITE HERITAGE SOCIETY OF POWELL RIVER', '138959549'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2185593, 'topic_source_id': 'S0066583', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['LEARNING LEGACY FUND SOCIETY', '737148296'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 2169877, 'topic_source_id': 'S0050148', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['LAKES LEGACY FUND SOCIETY'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 2168258, '

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(dam~ AND Investments~ AND Husky~)
[]
https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(dam~ OR Investments~ OR Husky~)
[{'id': 91605, 'topic_source_id': 'BC0090612', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['HUSKY INVESTMENTS LTD.'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 565788, 'topic_source_id': 'BC0550300', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['HUSKYHOWL VENTURES LTD.'], 'topic_inactive': True, 'topic_revoked': False}, {'id': 834011, 'topic_source_id': 'FM0473045', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['HUSKEY MEDIA'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 834030, 'topic_source_id': 'FM0473145', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['HUSKEY AUTO GROUP'], 'topic_inactive': False, 'topic_revoked': False}, {'id': 833929, 'topic_source_id': 'BC0817752', 'topic_issuer_id': 1, 'topic_type_id': 1, 'topic_names': ['HUSKEY HOLDINGS

In [54]:
dd.to_csv("orgbook_api_response_changed_full_together.csv")

In [6]:
import requests

import pprint
name = '(mmdonaldd~ AND restaunart~)' 
req = 'https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q='+name#+'"'
#req = 'https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q="dollarrama store"~'
response = requests.get(req)
print(req)
if len(response.json()) > 0:
    print(response.json()[0]['topic_names'])
    pprint.pprint(response.json())
else:
    print(response.json())

https://orgbook.gov.bc.ca/api/v4/search/fuzzy?q=(mmdonaldd~ AND restaunart~)
["MCDONALD'S RESTAURANTS"]
[{'id': 1397373,
  'topic_inactive': False,
  'topic_issuer_id': 1,
  'topic_names': ["MCDONALD'S RESTAURANTS"],
  'topic_revoked': False,
  'topic_source_id': 'FM0006039',
  'topic_type_id': 1},
 {'id': 1334434,
  'topic_inactive': False,
  'topic_issuer_id': 1,
  'topic_names': ["MCDONALD'S RESTAURANTS OF CANADA LIMITED/LES RESTAURANTS "
                  'MCDONALD DU CANADA LIMITÉE',
                  '120907092'],
  'topic_revoked': False,
  'topic_source_id': 'A0073512',
  'topic_type_id': 1},
 {'id': 1267108,
  'topic_inactive': True,
  'topic_issuer_id': 1,
  'topic_names': ["MCDONALD'S RESTAURANTS LANDS LIMITED"],
  'topic_revoked': False,
  'topic_source_id': 'A0010937',
  'topic_type_id': 1},
 {'id': 1290743,
  'topic_inactive': True,
  'topic_issuer_id': 1,
  'topic_names': ["MCDONALD'S RESTAURANTS OF CANADA LIMITED"],
  'topic_revoked': False,
  'topic_source_id': 'A00334

In [11]:
g = 'mmdonals restuarn'
l ='~ AND '.join(g.split())
t = '(' + l + '~)'
print(t)

(mmdonals~ AND restuarn~)
